CREAMOS LOS DF GAME Y REVIEW CON LOS QUE REALIZAREMOS EL SISTEMA DE RECOMENDACION

In [1]:
import pandas as pd
import json
import ast

In [2]:
rows = []
with open('Datasets/steam_games.json', 'r') as f:
        for line in f.readlines():
            data = json.loads(line)
            rows.append(data)

dfgames = pd.DataFrame(rows)
dfgames.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
rows2 = []
with open('Datasets/user_reviews.json', 'rt',encoding= 'utf-8') as f:
    for line in f.readlines():
        rows2.append(ast.literal_eval(line))
dfreviews = pd.DataFrame(rows2)
dfreviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [4]:
dfgames.dropna(how='all', inplace=True)
dfreviews.dropna(how='all', inplace=True)

In [5]:
dfgames=dfgames[['id','genres']]
dfgames.info(10)

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      32133 non-null  object
 1   genres  28852 non-null  object
dtypes: object(2)
memory usage: 753.2+ KB


In [6]:
dfreviews = dfreviews.explode('reviews')
dfreviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."


In [7]:
dfreviews['item_id'] = dfreviews['reviews'].apply(lambda x: x.get('item_id') if isinstance(x, dict) else '')
dfreviews['recommend'] = dfreviews['reviews'].apply(lambda x: x.get('recommend') if isinstance(x, dict) else '')
dfreviews['review'] = dfreviews['reviews'].apply(lambda x: x.get('review') if isinstance(x, dict) else '')
dfreviews.head(5)

,user_id,user_url,reviews,item_id,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",1250,True,Simple yet with great replayability. In my opi...
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",22200,True,It's unique and worth a playthrough.
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",43110,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",251610,True,I know what you think when you see this title ...
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",227300,True,For a simple (it's actually not all that simpl...


In [8]:
dfreviews=dfreviews[['user_id','item_id','recommend','review']]
dfreviews.head()

,user_id,item_id,recommend,review
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...
0,76561197970982479,22200,True,It's unique and worth a playthrough.
0,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,251610,True,I know what you think when you see this title ...
1,js41637,227300,True,For a simple (it's actually not all that simpl...


In [9]:
dfreviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59333 entries, 0 to 25798
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59333 non-null  object
 1   item_id    59333 non-null  object
 2   recommend  59333 non-null  object
 3   review     59333 non-null  object
dtypes: object(4)
memory usage: 2.3+ MB


In [10]:
df_merged = dfreviews.merge(dfgames, left_on='item_id', right_on='id', how='left')
df_merged.head()

,user_id,item_id,recommend,review,id,genres
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,1250,[Action]
1,76561197970982479,22200,True,It's unique and worth a playthrough.,22200,"[Action, Indie]"
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,NaN,NaN
3,js41637,251610,True,I know what you think when you see this title ...,NaN,NaN
4,js41637,227300,True,For a simple (it's actually not all that simpl...,227300,"[Indie, Simulation]"


In [11]:
df_merged = df_merged.dropna(subset=['id', 'genres'])
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50451 entries, 0 to 59332
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    50451 non-null  object
 1   item_id    50451 non-null  object
 2   recommend  50451 non-null  object
 3   review     50451 non-null  object
 4   id         50451 non-null  object
 5   genres     50451 non-null  object
dtypes: object(6)
memory usage: 2.7+ MB


In [12]:
from textblob import TextBlob
# Función para realizar el análisis de sentimiento
def sentiment_analysis(text):
    if isinstance(text, str):
        analysis = TextBlob(text)
        polarity = analysis.sentiment.polarity
        if polarity < 0:
            return 0  # Sentimiento negativo
        elif polarity == 0:
            return 1  # Sentimiento neutral
        else:
            return 2  # Sentimiento positivo
    else:
        return 1  # Valor por defecto para valores nulos o no strings

# Aplicar la función a la columna 'review' y crear la nueva columna 'sentiment_analysis'
df_merged['sentiment_analysis'] = df_merged['review'].apply(sentiment_analysis)

In [13]:
# eliminamos columnas que no usaremos
df_merged.drop(columns=['user_id','review','id'], inplace=True)
df_merged.head()


,item_id,recommend,genres,sentiment_analysis
0,1250,True,[Action],2
1,22200,True,"[Action, Indie]",2
4,227300,True,"[Indie, Simulation]",0
5,239030,True,"[Adventure, Indie]",0
6,248820,True,"[Action, Indie, RPG]",2


In [14]:
df_SimilitudCoseno= df_merged.copy()

In [15]:
df_SimilitudCoseno['item_id'] = df_SimilitudCoseno['item_id'].astype(int)

SIMILITUD DE COSENO

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
# Convierte la columna 'genres' en una representación one-hot
genres_encoded = pd.get_dummies(df_SimilitudCoseno['genres'].apply(pd.Series).stack(), prefix='genres')
genres_encoded = genres_encoded.groupby(level=0).sum()

In [18]:
genres_encoded.head()

,genres_Action,genres_Adventure,genres_Animation &amp; Modeling,genres_Audio Production,genres_Casual,genres_Design &amp; Illustration,genres_Early Access,genres_Education,genres_Free to Play,genres_Indie,genres_Massively Multiplayer,genres_RPG,genres_Racing,genres_Simulation,genres_Software Training,genres_Sports,genres_Strategy,genres_Utilities,genres_Video Production,genres_Web Publishing
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
5,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0


In [19]:
# Agrega la representación one-hot de los géneros al DataFrame original
df_SimilitudCoseno = pd.concat([df_SimilitudCoseno, genres_encoded], axis=1)

In [20]:
df_SimilitudCoseno.to_csv('df_SimilitudCoseno.csv', index=False)
dfreducido1 = pd.read_csv('df_SimilitudCoseno.csv')
dfreducido = dfreducido1.drop_duplicates(subset='item_id')
dfreducido.to_csv('df_SistemaRecomendacion2.csv', index=False)

FUNCION RECOMENDACION JUEGO

In [22]:
def recomendacion_juego(id_producto):
    import pandas as pd
    from sklearn.metrics.pairwise import cosine_similarity
    # Cargar el DataFrame desde el archivo CSV
    df = pd.read_csv('df_SistemaRecomendacion2.csv')
    num_recomendaciones=5
    
    # Calcula la similitud de coseno entre juegos
    features = df[['recommend', 'sentiment_analysis'] + list(df.columns[df.columns.str.startswith('genres_')])]
    similarities = cosine_similarity(features)
    
    # Mapea los índices de los juegos a sus nombres (item_id)
    index_to_item = {i: item_id for i, item_id in enumerate(df['item_id'])}
    
    # Encuentra el índice del juego dado su ID
    juego_index = df[df['item_id'] == id_producto].index[0]
    
    # Obtiene las similitudes de coseno del juego con otros juegos
    juego_similarities = similarities[juego_index]
    
    # Encuentra los índices de los juegos más similares (excluyendo el propio juego)
    juegos_similares_indices = juego_similarities.argsort()[-num_recomendaciones-1:-1][::-1]
    
    # Obtiene los nombres de los juegos recomendados
    juegos_recomendados = [index_to_item[i] for i in juegos_similares_indices]
    
    return juegos_recomendados

juego_recomendado = recomendacion_juego(440)
print("Juegos recomendados para el juego con ID 440:", juego_recomendado)

Juegos recomendados para el juego con ID 440: [206210, 386360, 372540, 239660, 346120]
